<a href="https://colab.research.google.com/github/welting82/AIclass/blob/main/titanic/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from urllib.request import urlretrieve
url = "https://github.com/welting82/AIclass/raw/main/titanic/test.csv"
urlretrieve(url, "test.csv")
url = "https://github.com/welting82/AIclass/raw/main/titanic/train.csv"
urlretrieve(url, "train.csv")

('train.csv', <http.client.HTTPMessage at 0x7f9a9543d1d0>)

In [18]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [19]:
datas = pd.concat([train,test],axis=0,ignore_index=True)
datas = datas.drop(["PassengerId","Survived"],axis=1)


In [20]:
#demo
name = "Braund, Mr. Owen Harris"
name.split(",")[-1].split(".")[0].strip()

'Mr'

In [21]:
def nameflow(name):
  return name.split(",")[-1].split(".")[0].strip()
datas["Name"] = datas["Name"].apply(nameflow)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,Dona,female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S


In [22]:
def cabinflow(cabin):
    if pd.isna(cabin):
        return cabin
    else:
        return cabin[0]
datas["Cabin"] = datas["Cabin"].apply(cabinflow)
datas["Cabin"].value_counts()

C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: Cabin, dtype: int64

In [23]:
dic = datas["Ticket"].value_counts()
def ticketflow(ticket):
  if pd.isna(ticket):
    return ticket
  else:
    return dic[ticket]
datas["Ticket"] = datas["Ticket"].apply(ticketflow)

CA. 2343    11
CA 2144      8
1601         8
3101295      7
PC 17608     7
            ..
347471       1
315086       1
229236       1
29107        1
28206        1
Name: Ticket, Length: 929, dtype: int64

In [28]:
s = datas.isna().sum()
s[s != 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [ ]:
#類別型態補值
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [29]:
#數值型態補值
med = datas.median().drop(["Pclass"])
datas.fillna(med)

Pclass     3.0000
Age       28.0000
SibSp      0.0000
Parch      0.0000
Ticket     1.0000
Fare      14.4542
dtype: float64